## SquamataGridMerge - Jupyter notebook used for preprocessing Geophysical data to be merged using Intrepid GridMerge and Geosoft Oasis montaj

### Created June 17, 2019, contact Phil Brown (pbrown@usgs.gov)

This module performs the following operations:
- Create list of data files to be processed.
- Create Geosoft batch file to reproject and grid data to be merged


In [1]:
# Load required Libraries
import sys
import os
import zipfile
import csv
#import pysb
import requests
import shutil
from shutil import copyfile
import zipfile
import datetime
import glob
from lxml import etree
import json
import pickle
import shutil
import fileinput
import json
import pandas as pd
import numpy as np
from IPython.core.display import display
from IPython.core.display import HTML
from lxml import etree
##from pymdwizard.core.xml_utils import XMLRecord
##from pymdwizard.core.xml_utils import XMLNode
import re
from ipywidgets import *
from IPython.display import display
from IPython.html.widgets import widgets
import datetime
import dateutil.parser
import time
from IPython.display import Javascript

C:\Users\pbrown\AppData\Local\Continuum\anacondaNew\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
#Set Input path, Geosoft Database Path of data to be preprocessed
dbInDataPath = r"C:\CurrentWork\MRAT\MT_ID_WY_GeophysicsProcessing\Mag" #The 'r' signifies a string literal. Use for paths.
# Set Output path, Geosoft Grid Path of data to be preprocessed
grdOutDataPath = r"C:\ID_WY_MT-Merge\Geosoft\Grids"

In [17]:
#Produce directory listing gd files
#mtDataDirList = os.walk(mtDataPath)
#mtDataDirList = [entry.path for entry in os.scandir(mtDataPath) if entry.is_dir()]
#dbInDataList = next(os.walk(dbInDataPath))[1]
dbInList = glob.glob(os.path.join(dbInDataPath, '*.gdb'),  recursive=True)
#dbInList[1]
fileName = []
fileListing = ''
# Get file name so we can rename the file later
for i in range(len(dbInList)):
  splitdbInList = dbInList[i].split('\\')
  fileName.append(splitdbInList[len(splitdbInList) - 1])
  #print (txtListFormatted[i])  
  fileListing = fileListing + splitdbInList[len(splitdbInList) - 1] + ','
print (fileListing)
fileName

id0538.gdb,id3013_new.gdb,ID_0609.gdb,ID_0656.gdb,ID_0657.gdb,ID_0667.gdb,ID_0671.gdb,ID_0709.gdb,ID_3008.gdb,ID_3015A.gdb,MT3021B.gdb,mt3149.gdb,MT_0085A.gdb,MT_0207.gdb,MT_0210.gdb,MT_0556.gdb,MT_0610.gdb,MT_0628.gdb,MT_0653.gdb,MT_0691.gdb,mt_3009.gdb,WY_0508.gdb,WY_0555.gdb,WY_0608.gdb,WY_0670.gdb,


['id0538.gdb',
 'id3013_new.gdb',
 'ID_0609.gdb',
 'ID_0656.gdb',
 'ID_0657.gdb',
 'ID_0667.gdb',
 'ID_0671.gdb',
 'ID_0709.gdb',
 'ID_3008.gdb',
 'ID_3015A.gdb',
 'MT3021B.gdb',
 'mt3149.gdb',
 'MT_0085A.gdb',
 'MT_0207.gdb',
 'MT_0210.gdb',
 'MT_0556.gdb',
 'MT_0610.gdb',
 'MT_0628.gdb',
 'MT_0653.gdb',
 'MT_0691.gdb',
 'mt_3009.gdb',
 'WY_0508.gdb',
 'WY_0555.gdb',
 'WY_0608.gdb',
 'WY_0670.gdb']

In [ ]:
## Create editable gs file.  
## Example text is created after running this cell
## This text is displayed by running "display(keywords) below
gsText = widgets.Textarea(
    value='CURRENT        Map,"c:\id_wy_mt-merge\Geosoft\IdWyMtMerged.map"\n' \
    + 'CURRENT        Database,"' + dbInList[i] + '"' \
    + 'SETINI         NEWXY.OLDX="UTM_X"' \
    + 'SETINI         NEWXY.OLDY="UTM_Y"' \
    + 'SETINI         NEWXY.USE_INPUT_XY=""' \
    + 'SETINI         NEWXY.OLDX="UTM_X"' \
    + 'SETINI         NEWXY.OLDY="UTM_Y"' \
    + 'SETINI         NEWXY.PROCESS="All lines/groups"' \
    + 'SETINI         IPJSET.S1GCS="\"WGS 84\""' \
    + 'SETINI         IPJSET.S2GCS="\"WGS 84\",6378137,0.0818191908426215,0"' \
    + 'SETINI         IPJSET.S3GCS=""' \
    + 'SETINI         IPJSET.S4GCS="dega,1"' \
    + 'SETINI         IPJSET.S5GCS="\"WGS 84\",0,0,0,0,0,0,0"' \
    + 'SETINI         IPJSET.S1PCS="\"NAD27 / UTM zone 12N\""' \
    + 'SETINI         IPJSET.S2PCS="NAD27,6378206.4,0.08227185422,0"' \
    + 'SETINI         IPJSET.S4PCS="m,1"' \
    + 'SETINI         IPJSET.S5PCS=""' \    
    + 'SETINI         NEWXY.NEWX="LonWGS84"' \ 
    + 'SETINI         NEWXY.NEWY="LatWGS84"' \
    + 'SETINI         IPJSET.S1GCS="\"WGS 84\""' \
    + 'SETINI         IPJSET.S2GCS="\"WGS 84\",6378137,0.0818191908426215,0"' \
    + 'SETINI         IPJSET.S3GCS=""' \
    + 'SETINI         IPJSET.S4GCS="dega,1"' \
    + 'SETINI         IPJSET.S5GCS="\"WGS 84\",0,0,0,0,0,0,0"' \
    + 'SETINI         IPJSET.S1PCS="\"NAD27 / UTM zone 12N\""' \
    + 'SETINI         IPJSET.S2PCS="NAD27,6378206.4,0.08227185422,0"' \
    + 'SETINI         IPJSET.S3PCS="\"Transverse Mercator\",0,-111,0.9996,500000,0"' \
    + 'SETINI         IPJSET.S4PCS="m,1"' \
    + 'SETINI         IPJSET.S5PCS=""' \
    + 'SETINI         NEWXY.OLDIPJ_NAME="\"NAD27 / UTM zone 12N\""' \
    + 'SETINI         NEWXY.OLDIPJ_DATUM="NAD27,6378206.4,0.08227185422,0"' \
    + 'SETINI         NEWXY.OLDIPJ_METHOD="\"Transverse Mercator\",0,-111,0.9996,500000,0"' \
    + 'SETINI         NEWXY.OLDIPJ_UNITS="m,1"' \
    + 'SETINI         NEWXY.OLDIPJ_LOCALDATUM=""' \
    + 'SETINI         NEWXY.NEWIPJ_NAME="\"WGS 84\""' \
    + 'SETINI         NEWXY.NEWIPJ_DATUM="\"WGS 84\",6378137,0.0818191908426215,0"' \
    + 'SETINI         NEWXY.NEWIPJ_METHOD=""' \
    + 'SETINI         NEWXY.NEWIPJ_UNITS="dega,1"' \
    + 'SETINI         NEWXY.NEWIPJ_LOCALDATUM="\"WGS 84\",0,0,0,0,0,0,0"' \
    + 'SETINI         NEWXY.WARPED="No"' \
    + 'SETINI         NEWXY.TRANSFORM_ACROSS_LOCAL_DATUM="No"' \
    + 'GX             newxy.gx' \
    + 'CURRENT        Map,"c:\id_wy_mt-merge\Geosoft\IdWyMtMerged.map"' \
    + 'CURRENT        Database,"C:\CurrentWork\MRAT\MT_ID_WY_GeophysicsProcessing\Mag\WY_0508.gdb"' \
    + 'SETINI         XYSET.X="UTM_X"' \
    + 'SETINI         XYSET.Y="UTM_Y"' \
    + 'SETINI         XYSET.Z="(none)"' \
    + 'SETINI         XYSET.X="LonWGS84"' \
    + 'SETINI         XYSET.Y="LatWGS84"' \
    + 'SETINI         XYSET.Z="(none)"' \
    + 'GX             xyset.gx' \
    + 'CURRENT        Map,"c:\id_wy_mt-merge\Geosoft\IdWyMtMerged.map"' \
    + 'CURRENT        Database,"' + dbInList[i] + '"' \
    + 'SETINI         RANGRID.LOCKED="0"' \
    + 'SETINI         RANGRID.RUNMODE="0"' \
    + 'SETINI         RANGRID.CHAN="mag"' \
    + 'SETINI         RANGRID.GRID=".\\WGS84_' + fileName[i] + '(GRD)"' \
    + 'SETINI         RANGRID.CS=""' \
    + 'GX             rangrid.gx',
    placeholder='Type something',
    #description='String:',
    layout=Layout(width='100%', height='666px'),
    disabled=False
)